<a href="https://colab.research.google.com/github/chu-ise/378A-2022/blob/main/notebooks/06/03_ngram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%pip install ekorpkit[tokenize]==0.1.12.post0.dev26

In [2]:
import gdown
import os
id = "1uDxxBNZ-qZscPjdEam0QEjTdKcFUBMuy"

data_file = "bok_minutes.csv"
gdown.download(id=id, output=data_file, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1uDxxBNZ-qZscPjdEam0QEjTdKcFUBMuy
To: /content/bok_minutes.csv
100%|██████████| 10.1M/10.1M [00:00<00:00, 153MB/s]


'bok_minutes.csv'

In [3]:
import pandas as pd
df = pd.read_csv(data_file)
df.text = df.text.astype(str)
df.head()

,id,filename,mdate,rdate,section,text
0,BOK_20181130_20181218_S1,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Economic Situation,일부 위원은 관련부서에서 지난 3/4분기 중 유로지역 경제성장 부진을 자동차 관련 ...
1,BOK_20181130_20181218_S2,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Foreign Currency,일부 위원은 그동안 글로벌펀드와 패시브펀드의 규모가 크게 확대되어 우리나라 자본유출...
2,BOK_20181130_20181218_S3,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Financial Markets,"일부 위원은 현재 대기업들이 전반적으로는 문제가 없지만, 건설 조선업 등에 속하는 ..."
3,BOK_20181130_20181218_S4,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Monetary Policy,일부 위원은 최근 경기상황과 금융불균형 등을 고려할 때 확장적 재정정책의 필요성에는...
4,BOK_20181130_20181218_S5,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Participants’ Views,일부 위원은 최근 실물경제 성장경로의 하방위험이 다소 커진 것으로 보이고 물가도 상...


In [4]:
from ekorpkit.preprocessors.tokenizer import MecabTokenizer

mecab = MecabTokenizer()

[ekorpkit]: Initializing mecab...)


In [20]:
corpus = df.text.to_list()
doc = mecab.tokenize_article(corpus[0]).split('\n')
len(doc)

76

# Build bigram dictionary

bigram_extractor 는 min_count 이상인 bigram 만을 선택하는 extractor

to_bigram 은 두 개의 list 를 zip 으로 묶어서 bigram list 을 만드는 함수

In [21]:
from collections import Counter

def bigram_extractor(sents, min_count=10):

    def to_bigram(tokens):
        bigrams = [(t0, t1) for t0, t1 in zip(tokens, tokens[1:])]
        return bigrams

    unigram_counter = Counter(word for sent in sents for word in sent.split())
    bigram_counter = Counter(
        (bigram for sent in sents for bigram in to_bigram(sent.split()))
    )

    bigram_dictionary = {
        bigram:count for bigram, count in bigram_counter.items()
        if count >= min_count and '/NN' in bigram[0]
    }

    def score(bigram, count):
        a = unigram_counter[bigram[0]]
        b = unigram_counter[bigram[1]]
        s = (count - min_count) / (a * b)
        return s

    bigram_score = {
        bigram:score(bigram, count) for bigram, count in bigram_dictionary.items()
    }

    return bigram_score

bigrams = bigram_extractor(doc)
len(bigrams)

18

In [22]:
sorted(bigrams.items(), key=lambda x:-x[1])[5:15]

[(('견해/NNG', '를/JKO'), 0.006617647058823529),
 (('것/NNB', '으로/JKB'), 0.005944931163954944),
 (('부서/NNG', '에서/JKB'), 0.005555555555555556),
 (('언급/NNG', '하/XSV'), 0.0032921810699588477),
 (('의견/NNG', '을/JKO'), 0.0015290519877675841),
 (('필요/NNG', '가/JKS'), 0.0012406947890818859),
 (('영향/NNG', '을/JKO'), 0.0011467889908256881),
 (('수/NNB', '/SP'), 0.00018722696068233826),
 (('최근/NNG', '/SP'), 0.0001594896331738437),
 (('경우/NNG', '/SP'), 0.00011961722488038278)]

# Bigram tokenizer

BigramTokenizer 는 base tokenizer 인 tagger 와 bigrams 를 입력받는 class

In [34]:
class BigramTokenizer:

    def __init__(self, bigrams, tokenize=lambda x:x.split()):
        self.bigrams = bigrams
        self.tokenize = tokenize

    def __call__(self, sent):
        if not sent:
            return []

        unigrams = self.tokenize(sent)

        bigrams = [(t0, t1) for t0, t1 in zip(unigrams, unigrams[1:])]
        bigrams = [bigram for bigram in bigrams if bigram in self.bigrams]
        bigrams = [f'{t0}-{t1}' for t0, t1 in bigrams]

        return unigrams + bigrams

bigram_tokenizer = BigramTokenizer(bigrams)

In [35]:
sent = doc[0]

bigram_tokenizer(sent) 로 클래스를 호출 

In [36]:
bigram_tokenizer(sent)

['일부/NNG',
 '/SP',
 '위원/NNG',
 '은/JX',
 '/SP',
 '관련/NNG',
 '부서/NNG',
 '에서/JKB',
 '/SP',
 '지난/VV+ETM',
 '/SP',
 '3/SN',
 '//SC',
 '4/SN',
 '분기/NNG',
 '/SP',
 '중/NNB',
 '/SP',
 '유로/NNG',
 '지역/NNG',
 '/SP',
 '경제/NNG',
 '성장/NNG',
 '/SP',
 '부진/NNG',
 '을/JKO',
 '/SP',
 '자동차/NNG',
 '/SP',
 '관련/NNG',
 '/SP',
 '규제/NNG',
 '/SP',
 '등/NNB',
 '/SP',
 '일시/NNG',
 '적/XSN',
 '/SP',
 '요인/NNG',
 '에/JKB',
 '/SP',
 '의한/VV+ETM',
 '/SP',
 '것/NNB',
 '으로/JKB',
 '/SP',
 '평가/NNG',
 '한/XSA+ETM',
 '/SP',
 '것/NNB',
 '과/JKB',
 '/SP',
 '관련/NNG',
 '하/XSV',
 '여/EC',
 ',/SC',
 '/SP',
 '동/MM',
 '/SP',
 '요인/NNG',
 '만/JX',
 '으로/JKB',
 '는/JX',
 '/SP',
 '성장/NNG',
 '률/XSN',
 '/SP',
 '둔화/NNG',
 '/SP',
 '설명/NNG',
 '에/JKB',
 '/SP',
 '충분/XR',
 '하/XSA',
 '지/EC',
 '/SP',
 '않/VX',
 '아/EC',
 '/SP',
 '보인다고/VV+EC',
 '/SP',
 '언급/NNG',
 '하/XSV',
 '였/EP',
 '음/ETN',
 './SF',
 '일부/NNG-/SP',
 '위원/NNG-은/JX',
 '관련/NNG-부서/NNG',
 '부서/NNG-에서/JKB',
 '등/NNB-/SP',
 '것/NNB-으로/JKB',
 '성장/NNG-률/XSN',
 '언급/NNG-하/XSV']

이를 CountVectorizer 의 tokenizer 로 입력

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=bigram_tokenizer)
x = vectorizer.fit_transform(doc)

In [28]:
x.shape

(76, 807)

# All bigrams

sklearn.feature_extraction.text.CountVectorizer 의 ngram_range=(1, 2) 로 설정한다면, 가능한 모든 종류의 bigram 을 term 으로 이용

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

def all_bigram_tokenizer(sent):
    if not sent:
        return []
    return sent.split()

vectorizer_ = CountVectorizer(
    ngram_range=(1,2),
    tokenizer=all_bigram_tokenizer
)

x_ = vectorizer_.fit_transform(doc)

In [32]:
x_.shape

(76, 2845)